In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import openpyxl
import matplotlib as plt
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import datetime
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [2]:
wfm = pd.read_excel(r'D:\\UF_Local_File\\MCM_2024\\Wimbledon_featured_matches.xlsx')

In [3]:
print(wfm["match_id"].unique())
for column_name in wfm.columns:
    print(column_name)
wfm

['2023-wimbledon-1301' '2023-wimbledon-1302' '2023-wimbledon-1303'
 '2023-wimbledon-1304' '2023-wimbledon-1305' '2023-wimbledon-1306'
 '2023-wimbledon-1307' '2023-wimbledon-1308' '2023-wimbledon-1309'
 '2023-wimbledon-1310' '2023-wimbledon-1311' '2023-wimbledon-1312'
 '2023-wimbledon-1313' '2023-wimbledon-1314' '2023-wimbledon-1315'
 '2023-wimbledon-1316' '2023-wimbledon-1401' '2023-wimbledon-1402'
 '2023-wimbledon-1403' '2023-wimbledon-1404' '2023-wimbledon-1405'
 '2023-wimbledon-1406' '2023-wimbledon-1407' '2023-wimbledon-1408'
 '2023-wimbledon-1501' '2023-wimbledon-1502' '2023-wimbledon-1503'
 '2023-wimbledon-1504' '2023-wimbledon-1601' '2023-wimbledon-1602'
 '2023-wimbledon-1701']
match_id
player1
player2
elapsed_time
set_no
game_no
point_no
p1_sets
p2_sets
p1_games
p2_games
p1_score
p2_score
server
serve_no
point_victor
p1_points_won
p2_points_won
game_victor
set_victor
p1_ace
p2_ace
p1_winner
p2_winner
winner_shot_type
p1_double_fault
p2_double_fault
p1_unf_err
p2_unf_err
p1_net_

,match_id,player1,player2,elapsed_time,set_no,game_no,point_no,p1_sets,p2_sets,p1_games,...,p2_break_pt_won,p1_break_pt_missed,p2_break_pt_missed,p1_distance_run,p2_distance_run,rally_count,speed_mph,serve_width,serve_depth,return_depth
0,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:00,1,1,1,0,0,0,...,0,0,0,6.000,7.840,2,95.0,BC,NCTL,ND
1,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:00:38,1,1,2,0,0,0,...,0,0,0,5.253,7.094,1,118.0,B,CTL,ND
2,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:01:01,1,1,3,0,0,0,...,0,0,0,13.800,19.808,4,120.0,B,NCTL,D
3,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:01:31,1,1,4,0,0,0,...,0,0,0,51.108,75.631,13,130.0,BW,CTL,D
4,2023-wimbledon-1301,Carlos Alcaraz,Nicolas Jarry,00:02:21,1,1,5,0,0,0,...,0,0,0,0.649,0.813,1,112.0,W,NCTL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7279,2023-wimbledon-1701,Carlos Alcaraz,Novak Djokovic,04:39:58,5,10,330,2,2,5,...,0,0,0,38.620,50.971,11,123.0,C,NCTL,D
7280,2023-wimbledon-1701,Carlos Alcaraz,Novak Djokovic,04:40:43,5,10,331,2,2,5,...,0,0,0,17.533,27.766,5,114.0,W,NCTL,ND
7281,2023-wimbledon-1701,Carlos Alcaraz,Novak Djokovic,04:41:20,5,10,332,2,2,5,...,0,0,0,10.882,18.328,4,124.0,W,NCTL,D
7282,2023-wimbledon-1701,Carlos Alcaraz,Novak Djokovic,04:41:52,5,10,333,2,2,5,...,0,0,0,7.893,10.537,1,130.0,C,NCTL,D


In [4]:
def convert_to_elapsed_time(timestamp):
    return timestamp.strftime('%H:%M:%S')  # Format datetime.time object to "hh:mm:ss"

# Apply the conversion function to the 'elapsed_time' column of your actual DataFrame 'wfm'
wfm['elapsed_time'] = wfm['elapsed_time'].apply(convert_to_elapsed_time)

# Find NaN values in the DataFrame 'wfm'
nan_locations = wfm.isna()

# Iterate over each column to identify NaN values and print match_id, column name, elapsed_time
for col in wfm.columns:
    for idx, is_nan in nan_locations[col].items():
        if is_nan:  # Check if the value at this location is NaN
            match_id = wfm.loc[idx, 'match_id']
            column_name = col
            if wfm.loc[idx+1, 'serve_no'] == 2:
                failed_first_serve = True
            else:
                failed_first_serve = False
            double_fault = True if wfm.loc[idx, 'p1_double_fault'] == wfm.loc[idx, 'p2_double_fault'] else False
            elapsed_time = wfm.loc[idx, 'elapsed_time'] if 'elapsed_time' in wfm.columns else None
            print(f"NaN found in Match ID: {match_id[-4:]}, Column Name: {column_name}, Elapsed Time: {elapsed_time}, Is failed first serve? {failed_first_serve}, Is double fault? {double_fault}")

NaN found in Match ID: 1301, Column Name: speed_mph, Elapsed Time: 00:32:57, Is failed first serve? False, Is double fault? False
NaN found in Match ID: 1301, Column Name: speed_mph, Elapsed Time: 02:06:20, Is failed first serve? False, Is double fault? False
NaN found in Match ID: 1301, Column Name: speed_mph, Elapsed Time: 02:14:53, Is failed first serve? False, Is double fault? False
NaN found in Match ID: 1301, Column Name: speed_mph, Elapsed Time: 02:28:21, Is failed first serve? False, Is double fault? False
NaN found in Match ID: 1301, Column Name: speed_mph, Elapsed Time: 02:53:10, Is failed first serve? False, Is double fault? False
NaN found in Match ID: 1301, Column Name: speed_mph, Elapsed Time: 02:55:10, Is failed first serve? True, Is double fault? False
NaN found in Match ID: 1301, Column Name: speed_mph, Elapsed Time: 03:05:04, Is failed first serve? False, Is double fault? True
NaN found in Match ID: 1301, Column Name: speed_mph, Elapsed Time: 03:13:12, Is failed first

In [5]:
match_dfs = {}
match_ids = wfm["match_id"].unique()
for match_id in match_ids:
    match_dfs[match_id] = wfm[wfm["match_id"] == match_id]

match_dfs

{'2023-wimbledon-1301':                 match_id         player1        player2 elapsed_time  set_no  \
 0    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:00       1   
 1    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:38       1   
 2    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:01       1   
 3    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:31       1   
 4    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:02:21       1   
 ..                   ...             ...            ...          ...     ...   
 295  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:53:25       4   
 296  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:54:01       4   
 297  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:02       4   
 298  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:34       4   
 299  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:56:11       4   
 
   

In [6]:
raw_match_dfs = match_dfs   # Save a raw data copy of the original dataframe

def time_to_minutes(time_str):
    hours, minutes, seconds = map(int, time_str.split(':'))
    total_minutes = hours * 60 + minutes + seconds / 60.0
    return total_minutes

for match_id in match_ids:
# Here you can modify values in each matches
    match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
    
    match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
    
    match_dfs[match_id].loc[:,'double_fault'] = match_dfs[match_id].loc[:,'p1_double_fault']-match_dfs[match_id].loc[:,'p2_double_fault']
    
    match_dfs[match_id].loc[:,'p1_break_pt_result'] = match_dfs[match_id].loc[:,'p1_break_pt_won']-match_dfs[match_id].loc[:,'p1_break_pt_missed']
    match_dfs[match_id].loc[:,'p2_break_pt_result'] = match_dfs[match_id].loc[:,'p2_break_pt_won']-match_dfs[match_id].loc[:,'p2_break_pt_missed']
    
    match_dfs[match_id].loc[:,'elapsed_minutes'] = match_dfs[match_id].loc[:,'elapsed_time'].apply(time_to_minutes)  # Currently not used
    
    match_dfs[match_id].loc[:,'p1_consistency'] = 0
    match_dfs[match_id].loc[:,'p2_consistency'] = 0
    for index, row in match_dfs[match_id].iterrows():
        if index<=3:
            p1_consistency_count = 0
            p2_consistency_count = 0
        else:
            p1_consistency_count = match_dfs[match_id].loc[index-3:index, 'point_victor'].eq(1).sum()
            p2_consistency_count = match_dfs[match_id].loc[index-3:index, 'point_victor'].eq(2).sum()
        match_dfs[match_id].loc[index, 'p1_consistency'] = p1_consistency_count
        match_dfs[match_id].loc[index, 'p2_consistency'] = p2_consistency_count
match_dfs

C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
C:\Users\zhuzi\AppData\Local\Temp\ipy

C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
C:\Users\zhuzi\AppData\Local\Temp\ipy

C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
C:\Users\zhuzi\AppData\Local\Temp\ipy

C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
C:\Users\zhuzi\AppData\Local\Temp\ipy

C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
C:\Users\zhuzi\AppData\Local\Temp\ipy

C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
C:\Users\zhuzi\AppData\Local\Temp\ipy

C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
C:\Users\zhuzi\AppData\Local\Temp\ipy

C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
C:\Users\zhuzi\AppData\Local\Temp\ipy

C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'cumulative_points_diff'] = match_dfs[match_id].loc[:,'p1_points_won']-match_dfs[match_id].loc[:,'p2_points_won']
C:\Users\zhuzi\AppData\Local\Temp\ipykernel_13892\3353753656.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_dfs[match_id].loc[:,'ace'] = match_dfs[match_id].loc[:,'p1_ace']-match_dfs[match_id].loc[:,'p2_ace']
C:\Users\zhuzi\AppData\Local\Temp\ipy

{'2023-wimbledon-1301':                 match_id         player1        player2 elapsed_time  set_no  \
 0    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:00       1   
 1    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:38       1   
 2    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:01       1   
 3    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:31       1   
 4    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:02:21       1   
 ..                   ...             ...            ...          ...     ...   
 295  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:53:25       4   
 296  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:54:01       4   
 297  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:02       4   
 298  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:34       4   
 299  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:56:11       4   
 
   

In [7]:
test_match = match_dfs.pop('2023-wimbledon-1701')
print("train match\n", match_dfs,"\ntest match\n", test_match)

train match
 {'2023-wimbledon-1301':                 match_id         player1        player2 elapsed_time  set_no  \
0    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:00       1   
1    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:00:38       1   
2    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:01       1   
3    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:01:31       1   
4    2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     00:02:21       1   
..                   ...             ...            ...          ...     ...   
295  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:53:25       4   
296  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:54:01       4   
297  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:02       4   
298  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:55:34       4   
299  2023-wimbledon-1301  Carlos Alcaraz  Nicolas Jarry     03:56:11       4   

  

In [8]:
# Initialize lists to store the extracted data
X_classification_list = []
y_classification_list = []
accuracy_sum = 0


# Iterate over each match DataFrame in the match_dfs dictionary
for key, df in match_dfs.items():
    
    # Select relevant columns for classification
    X_classification = df[[
                           'cumulative_points_diff',
                           'ace',
                           'double_fault',
                           'p1_break_pt_result', 'p2_break_pt_result',
                           'p1_consistency', 'p2_consistency',
                           'p1_unf_err', 'p2_unf_err'
                           ]]
    y_classification = df['point_victor']  # Target variable (player 1 or player 2 win)

    # Split data into train and test sets
    X_train_tree, X_test_tree, y_train_tree, y_test_tree = train_test_split(X_classification, y_classification, test_size=0.2, shuffle=False)

    # Train Decision Tree Classifier
    clf = DecisionTreeClassifier()
    clf.fit(X_train_tree, y_train_tree)

    # Evaluate Decision Tree Classifier
    y_pred_class = clf.predict(X_test_tree)
    accuracy = accuracy_score(y_test_tree, y_pred_class)
    print(f"Accuracy for {key}: {accuracy}")
    accuracy_sum += accuracy

    # Store features and labels for concatenation
    X_classification_list.append(X_classification)
    y_classification_list.append(y_classification)

accuracy_average = accuracy_sum/31
print(f'The average accuracy in training sets is: {accuracy_average}')
# Concatenate features and labels from all matches
X_classification_concat = pd.concat(X_classification_list)
y_classification_concat = pd.concat(y_classification_list)

# Train final Decision Tree Classifier on concatenated data
final_clf = DecisionTreeClassifier()
final_clf.fit(X_classification_concat, y_classification_concat)


Accuracy for 2023-wimbledon-1301: 0.7166666666666667
Accuracy for 2023-wimbledon-1302: 0.7073170731707317
Accuracy for 2023-wimbledon-1303: 0.7037037037037037
Accuracy for 2023-wimbledon-1304: 0.7058823529411765
Accuracy for 2023-wimbledon-1305: 0.7
Accuracy for 2023-wimbledon-1306: 0.7910447761194029
Accuracy for 2023-wimbledon-1307: 0.5957446808510638
Accuracy for 2023-wimbledon-1308: 0.5526315789473685
Accuracy for 2023-wimbledon-1309: 0.7674418604651163
Accuracy for 2023-wimbledon-1310: 0.78125
Accuracy for 2023-wimbledon-1311: 0.9117647058823529
Accuracy for 2023-wimbledon-1312: 0.8363636363636363
Accuracy for 2023-wimbledon-1313: 0.8275862068965517
Accuracy for 2023-wimbledon-1314: 0.5405405405405406
Accuracy for 2023-wimbledon-1315: 0.85
Accuracy for 2023-wimbledon-1316: 0.8529411764705882
Accuracy for 2023-wimbledon-1401: 0.7111111111111111
Accuracy for 2023-wimbledon-1402: 0.7818181818181819
Accuracy for 2023-wimbledon-1403: 0.76
Accuracy for 2023-wimbledon-1404: 0.75862068965

DecisionTreeClassifier()

In [9]:
test_match_cleaned = test_match[[
                           'cumulative_points_diff',
                           'ace',
                           'double_fault',
                           'p1_break_pt_result', 'p2_break_pt_result',
                           'p1_consistency', 'p2_consistency',
                           'p1_unf_err', 'p2_unf_err'
                           ]]
test_predictions = final_clf.predict(test_match_cleaned)
print("Test Predictions:", test_predictions)

y_test_classification = test_match['point_victor']
accuracy = accuracy_score(y_test_classification, test_predictions)
conf_matrix = confusion_matrix(y_test_classification, test_predictions)

print("Accuracy on Test Match:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

Test Predictions: [2 1 1 2 1 2 2 2 2 2 2 2 2 1 1 2 2 1 2 2 1 2 2 2 2 2 1 1 1 2 2 2 2 2 2 1 1
 2 2 1 1 1 1 2 2 2 2 2 1 1 1 1 1 1 2 2 1 2 1 1 2 1 1 2 1 2 2 2 1 2 2 1 1 2
 2 2 2 2 1 1 1 2 1 2 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 2 2 2 2 1 1 1 1 2 1 2 2
 2 2 2 2 2 1 1 1 1 1 2 1 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 2 2 1 2 2 1 1 2
 1 1 1 2 1 2 2 1 2 2 2 1 2 2 2 1 1 1 1 1 1 2 2 2 1 2 1 2 1 1 2 2 1 1 2 2 2
 1 2 2 1 1 2 1 2 1 2 2 2 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 2 1 1 1 1 1 1 2 2 2
 2 2 2 1 1 1 2 1 2 1 2 1 1 1 2 1 1 1 1 1 2 1 2 1 2 2 1 2 1 1 2 2 2 2 2 2 2
 2 2 2 1 1 1 2 2 2 2 2 2 1 2 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 1 1 1 1 1 1 2 1
 1 1 1 1 1 1 2 2 1 2 2 2 2 2 1 1 1 1 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 1
 1]
Accuracy on Test Match: 0.7934131736526946
Confusion Matrix:
 [[130  38]
 [ 31 135]]


In [10]:
# Split data into train and test sets
X_train_forest, X_test_forest, y_train_forest, y_test_forest = train_test_split(X_classification, y_classification, test_size=0.05, shuffle=False)

# Initialize Random Forest classifier with default parameters
rf_clf = RandomForestClassifier()

# Train the classifier on the training data
rf_clf.fit(X_train_forest, y_train_forest)

# Make predictions on the test data
y_pred = rf_clf.predict(test_match_cleaned)
print(y_pred)

# Calculate accuracy score
accuracy = accuracy_score(test_match['point_victor'], y_pred)
print("Accuracy:", accuracy)

[2 1 1 1 1 2 2 1 2 1 2 2 2 1 1 2 2 1 1 2 1 2 2 2 2 2 1 1 2 2 1 2 1 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 2 2 2 1 2 1 1 2 1 1 2 1 2 1 2 1 2 2 2 2 2
 2 2 2 2 1 1 1 2 2 2 2 1 1 1 1 2 2 2 2 2 1 1 1 1 2 2 2 2 2 1 1 1 1 2 2 2 2
 2 2 2 2 2 2 1 1 1 1 2 1 2 1 2 2 2 2 1 1 1 1 1 2 2 1 1 1 1 2 2 2 2 2 1 1 2
 1 1 1 2 1 2 1 1 1 2 2 1 2 2 2 1 1 1 1 1 1 2 2 2 1 2 1 2 1 1 2 2 1 1 2 2 1
 1 2 2 1 2 2 1 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 2 2 2
 2 2 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1 2 2 1 2 1 1 2 2 2 2 2 1 2
 1 1 1 1 1 1 1 2 2 2 2 2 1 2 2 2 2 1 1 2 2 1 2 2 2 2 2 1 2 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 1 2 2 1 2 1 1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 2 2 2 2 1 1 1 1
 1]
Accuracy: 0.7904191616766467


In [14]:
# comparison between decision tree and decision tree group
consistancy = accuracy_score(test_predictions, y_pred)
print("Consistancy between two models are: ", consistancy)

Consistancy between two models are:  0.8293413173652695
